In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation
from matplotlib import image
from matplotlib.animation import FuncAnimation
import sympy as sp

Автоматизированные символьные вычисления

In [14]:
t = sp.symbols('t', real=True, function = True)

q = [sp.Function(f'q_{i}')(t) for i in range(4)]

p = [sp.Function(f'p_{i}')(t) for i in range(3)]

M_ext = sp.symbols('M^ext_:{}'.format(3))

M_airfit = sp.symbols('M^airfit_:{}_:{}'.format(4,3))

J_qq = sp.symbols('J^qq_:{}_:{}'.format(3,3))

J_helix = sp.symbols('J^helix_:{}_:{}_:{}'.format(4,3,3))

dp = [sp.Function(f'dp_{i}')(t) for i in range(3)]

w_qq = [sp.Function(f'w^qq_{i}')(t) for i in range(3)]

w_helix = sp.symbols('w^helix_:{}_:{}'.format(4,3))

F_grav = sp.symbols('F^grav_:{}'.format(3))

F_lift = sp.symbols('F^lift_:{}'.format(3))
"""масса квадрокоптера и ускорение силы тяжести"""
g, Mqq = sp.symbols('g M_qq')

In [15]:
R_q = sp.Matrix(3,3, lambda i, j: 0)
R_q[0,0] = q[0]**2 + q[1]**2 - q[2]**2 - q[3]**2 
R_q[0,1] = 2*(q[1]*q[2] - q[0]*q[3])
R_q[0,2] = 2*(q[0]*q[2] + q[1]*q[3])
R_q[1,0] = 2*(q[0]*q[3] + q[1]*q[2]) 
R_q[1,1] = q[0]**2 - q[1]**2 + q[2]**2 - q[3]**2 
R_q[1,2] = 2*(q[2]*q[3] - q[0]*q[1]) 
R_q[2,0] = 2*(q[1]*q[3] - q[0]*q[2]) 
R_q[2,1] =  2*(q[0]*q[1] + q[2]*q[3])
R_q[2,2] = q[0]**2 - q[1]**2 - q[2]**2 + q[3]**2 
R_q

Matrix([
[q_0(t)**2 + q_1(t)**2 - q_2(t)**2 - q_3(t)**2,            -2*q_0(t)*q_3(t) + 2*q_1(t)*q_2(t),             2*q_0(t)*q_2(t) + 2*q_1(t)*q_3(t)],
[            2*q_0(t)*q_3(t) + 2*q_1(t)*q_2(t), q_0(t)**2 - q_1(t)**2 + q_2(t)**2 - q_3(t)**2,            -2*q_0(t)*q_1(t) + 2*q_2(t)*q_3(t)],
[           -2*q_0(t)*q_2(t) + 2*q_1(t)*q_3(t),             2*q_0(t)*q_1(t) + 2*q_2(t)*q_3(t), q_0(t)**2 - q_1(t)**2 - q_2(t)**2 + q_3(t)**2]])

In [16]:
OmegaTen = sp.Matrix(4, 4, lambda i, j: 0)
for i in range(0, 4):
    for j in range(0, 4):
        if i < j:
            if i == 0:
                OmegaTen[i, j] = -w_qq[j-1]
            elif i == 1:
                OmegaTen[i, j] = -w_qq[1-j]
                OmegaTen[1, 3] = w_qq[1]
            else:
                OmegaTen[i, j] = -w_qq[0]
                
        elif i > j:
            OmegaTen[i, j] = -OmegaTen[j, i]
OmegaTen

Matrix([
[        0, -w^qq_0(t), -w^qq_1(t), -w^qq_2(t)],
[w^qq_0(t),          0, -w^qq_2(t),  w^qq_1(t)],
[w^qq_1(t),  w^qq_2(t),          0, -w^qq_0(t)],
[w^qq_2(t), -w^qq_1(t),  w^qq_0(t),          0]])

In [17]:
Flift = sp.Matrix(3,1, lambda i,j: F_lift[i])
Flift

Matrix([
[F^lift_0],
[F^lift_1],
[F^lift_2]])

In [18]:
Fgrav = sp.Matrix(3,1, lambda i,j: F_grav[i])
Fgrav

Matrix([
[F^grav_0],
[F^grav_1],
[F^grav_2]])

In [19]:
Fgrav = sp.Matrix(3,1, lambda i,j: F_grav[i])
Fgrav

Matrix([
[F^grav_0],
[F^grav_1],
[F^grav_2]])

In [20]:
Mext = sp.Matrix(3,1, lambda i,j: M_ext[i])
Mext

Matrix([
[M^ext_0],
[M^ext_1],
[M^ext_2]])

In [21]:
Mairfit = (sp.Matrix(3,1, lambda i,j: M_airfit[i]) + sp.Matrix(3,1, lambda i,j: M_airfit[i+3]) + 
           sp.Matrix(3,1, lambda i,j: M_airfit[i+6]) + sp.Matrix(3,1, lambda i,j: M_airfit[i+9]))
Mairfit

Matrix([
[M^airfit_0_0 + M^airfit_1_0 + M^airfit_2_0 + M^airfit_3_0],
[M^airfit_0_1 + M^airfit_1_1 + M^airfit_2_1 + M^airfit_3_1],
[M^airfit_0_2 + M^airfit_1_2 + M^airfit_2_2 + M^airfit_3_2]])

In [22]:
w_helex_0_dot = (sp.Matrix([sp.symbols('\dot{w}^{helex_0}_0'),
                          sp.symbols('\dot{w}^{helex_0}_1'),
                          sp.symbols('\dot{w}^{helex_0}_2')]))
w_helex_1_dot = sp.Matrix([sp.symbols('\dot{w}^{helex_1}_0'),
                            sp.symbols('\dot{w}^{helex_1}_1'),
                            sp.symbols('\dot{w}^{helex_1}_2')])
w_helex_2_dot = sp.Matrix([sp.symbols('\dot{w}^{helex_2}_0'),
                           sp.symbols('\dot{w}^{helex_2}_1'),
                           sp.symbols('\dot{w}^{helex_2}_2')])
w_helex_3_dot = sp.Matrix([sp.symbols('\dot{w}^{helex_3}_0'),
                           sp.symbols('\dot{w}^{helex_3}_1'),
                           sp.symbols('\dot{w}^{helex_3}_2')])

w_helex_3_dot + w_helex_2_dot + w_helex_1_dot + w_helex_0_dot

Matrix([
[\dot{w}^{helex_0}_0 + \dot{w}^{helex_1}_0 + \dot{w}^{helex_2}_0 + \dot{w}^{helex_3}_0],
[\dot{w}^{helex_0}_1 + \dot{w}^{helex_1}_1 + \dot{w}^{helex_2}_1 + \dot{w}^{helex_3}_1],
[\dot{w}^{helex_0}_2 + \dot{w}^{helex_1}_2 + \dot{w}^{helex_2}_2 + \dot{w}^{helex_3}_2]])

In [23]:
w_helix_0 = sp.Matrix(3,1, lambda i,j: w_helix[i])
w_helix_1 = sp.Matrix(3,1, lambda i,j: w_helix[i+3])
w_helix_2 = sp.Matrix(3,1, lambda i,j: w_helix[i+6])
w_helix_3 = sp.Matrix(3,1, lambda i,j: w_helix[i+9])
w_helix_0+w_helix_1+w_helix_2+w_helix_3

Matrix([
[w^helix_0_0 + w^helix_1_0 + w^helix_2_0 + w^helix_3_0],
[w^helix_0_1 + w^helix_1_1 + w^helix_2_1 + w^helix_3_1],
[w^helix_0_2 + w^helix_1_2 + w^helix_2_2 + w^helix_3_2]])

In [24]:
wqq = sp.Matrix(3,1, lambda i,j: w_qq[i])
wqq

Matrix([
[w^qq_0(t)],
[w^qq_1(t)],
[w^qq_2(t)]])

In [25]:
dpv = sp.Matrix(3,1, lambda i,j: p[i])
dpv

Matrix([
[p_0(t)],
[p_1(t)],
[p_2(t)]])

\begin{equation}
 \begin{cases}
 \widehat{J}_{qq}\dot{\vec{\omega}}_{qq} = \sum_{j} \vec{M}_{ext, j} + \vec{M}_{air lift} - \sum_{i} \widehat{J}_{helix,   i}\dot{\vec{\omega}}_{helix, i} - \vec{\omega}_{qq} \times (\widehat{J}_{qq}\vec{\omega}_{qq} + \sum_{i} \widehat{J}_{helix, i} \vec{\omega}_{helix, i}) \\
   \vec{\dot{q}} = \frac{1}{2}\widehat{\Omega}\vec{q}\\
   \vec{F}_{grav} + \widehat{R}(q(t))\vec{F}_{lift} + \vec{F}_{air} = \dot{\vec{p}}\\
 \end{cases}
\end{equation}

момент силы создаваемый одним из поднимающих винтов $\vec{M}_{air lift} = \vec{r}\times\ \vec{F}_{lift} $ 

для удобства сразу предполагаю вид всех тензоров инерции диагональным

In [26]:
Jqq =  sp.Matrix(3,3, lambda i,j: J_qq[3*i+j] if i == j else 0)
J_helix_0 = sp.Matrix(3,3, lambda i,j: J_helix[3*i+j] if i == j else 0)
J_helix_1 = sp.Matrix(3,3, lambda i,j: J_helix[3*i+j+9] if i == j else 0)
J_helix_2 = sp.Matrix(3,3, lambda i,j: J_helix[3*i+j+18] if i == j else 0)
J_helix_3 = sp.Matrix(3,3, lambda i,j: J_helix[3*i+j+27] if i == j else 0)

In [27]:
"""первоее векторное уравнение переписанное в явном виде"""
right_part_first_equation = (Jqq.inv()*(Mext + Mairfit - (J_helix_0*w_helex_0_dot + J_helix_1*w_helex_1_dot + 
                             J_helix_2*w_helex_2_dot + J_helix_3*w_helex_3_dot)))

In [28]:
right_part_first_equation = right_part_first_equation - sp.simplify(Jqq.inv()*(wqq.cross(Jqq*wqq + 
                            J_helix_0*w_helix_0 + J_helix_1*w_helix_1 + J_helix_2*w_helix_2 + J_helix_3*w_helix_3)))

In [29]:
sp.simplify(right_part_first_equation)

Matrix([
[(-J^helix_0_0_0*\dot{w}^{helex_0}_0 - J^helix_1_0_0*\dot{w}^{helex_1}_0 - J^helix_2_0_0*\dot{w}^{helex_2}_0 - J^helix_3_0_0*\dot{w}^{helex_3}_0 + M^airfit_0_0 + M^airfit_1_0 + M^airfit_2_0 + M^airfit_3_0 + M^ext_0 + (J^helix_0_1_1*w^helix_0_1 + J^helix_1_1_1*w^helix_1_1 + J^helix_2_1_1*w^helix_2_1 + J^helix_3_1_1*w^helix_3_1 + J^qq_1_1*w^qq_1(t))*w^qq_2(t) - (J^helix_0_2_2*w^helix_0_2 + J^helix_1_2_2*w^helix_1_2 + J^helix_2_2_2*w^helix_2_2 + J^helix_3_2_2*w^helix_3_2 + J^qq_2_2*w^qq_2(t))*w^qq_1(t))/J^qq_0_0],
[(-J^helix_0_1_1*\dot{w}^{helex_0}_1 - J^helix_1_1_1*\dot{w}^{helex_1}_1 - J^helix_2_1_1*\dot{w}^{helex_2}_1 - J^helix_3_1_1*\dot{w}^{helex_3}_1 + M^airfit_0_1 + M^airfit_1_1 + M^airfit_2_1 + M^airfit_3_1 + M^ext_1 - (J^helix_0_0_0*w^helix_0_0 + J^helix_1_0_0*w^helix_1_0 + J^helix_2_0_0*w^helix_2_0 + J^helix_3_0_0*w^helix_3_0 + J^qq_0_0*w^qq_0(t))*w^qq_2(t) + (J^helix_0_2_2*w^helix_0_2 + J^helix_1_2_2*w^helix_1_2 + J^helix_2_2_2*w^helix_2_2 + J^helix_3_2_2*w^helix_3_2 +

или в явноми виде от этого
\begin{equation}
 \dot{\vec{\omega}}_{qq} = \widehat{J}^{-1}_{qq}(\sum_{j} \vec{M}_{ext, j} + \vec{M}_{air lift} - \sum_{i} \widehat{J}_{helix,   i}\dot{\vec{\omega}}_{helix, i} - \vec{\omega}_{qq} \times (\widehat{J}_{qq}\vec{\omega}_{qq} + \sum_{i} \widehat{J}_{helix, i} \vec{\omega}_{helix, i})) 
\end{equation}

In [30]:
qq = sp.Matrix(4,1, lambda i,j: q[i])
qq

Matrix([
[q_0(t)],
[q_1(t)],
[q_2(t)],
[q_3(t)]])

In [31]:
right_part_second_equation = 1/2 * OmegaTen*qq
right_part_second_equation

Matrix([
[-0.5*q_1(t)*w^qq_0(t) - 0.5*q_2(t)*w^qq_1(t) - 0.5*q_3(t)*w^qq_2(t)],
[ 0.5*q_0(t)*w^qq_0(t) - 0.5*q_2(t)*w^qq_2(t) + 0.5*q_3(t)*w^qq_1(t)],
[ 0.5*q_0(t)*w^qq_1(t) + 0.5*q_1(t)*w^qq_2(t) - 0.5*q_3(t)*w^qq_0(t)],
[ 0.5*q_0(t)*w^qq_2(t) - 0.5*q_1(t)*w^qq_1(t) + 0.5*q_2(t)*w^qq_0(t)]])

In [32]:
right_part_three_equation = (Fgrav + R_q*Flift)
right_part_three_equation

Matrix([
[F^grav_0 + F^lift_0*(q_0(t)**2 + q_1(t)**2 - q_2(t)**2 - q_3(t)**2) + F^lift_1*(-2*q_0(t)*q_3(t) + 2*q_1(t)*q_2(t)) + F^lift_2*(2*q_0(t)*q_2(t) + 2*q_1(t)*q_3(t))],
[F^grav_1 + F^lift_0*(2*q_0(t)*q_3(t) + 2*q_1(t)*q_2(t)) + F^lift_1*(q_0(t)**2 - q_1(t)**2 + q_2(t)**2 - q_3(t)**2) + F^lift_2*(-2*q_0(t)*q_1(t) + 2*q_2(t)*q_3(t))],
[F^grav_2 + F^lift_0*(-2*q_0(t)*q_2(t) + 2*q_1(t)*q_3(t)) + F^lift_1*(2*q_0(t)*q_1(t) + 2*q_2(t)*q_3(t)) + F^lift_2*(q_0(t)**2 - q_1(t)**2 - q_2(t)**2 + q_3(t)**2)]])

In [33]:
right_part_our = sp.Matrix.vstack(right_part_first_equation, right_part_second_equation, right_part_three_equation)

In [34]:
right_part_our

Matrix([
[-(-(J^helix_0_1_1*w^helix_0_1 + J^helix_1_1_1*w^helix_1_1 + J^helix_2_1_1*w^helix_2_1 + J^helix_3_1_1*w^helix_3_1 + J^qq_1_1*w^qq_1(t))*w^qq_2(t) + (J^helix_0_2_2*w^helix_0_2 + J^helix_1_2_2*w^helix_1_2 + J^helix_2_2_2*w^helix_2_2 + J^helix_3_2_2*w^helix_3_2 + J^qq_2_2*w^qq_2(t))*w^qq_1(t))/J^qq_0_0 + (-J^helix_0_0_0*\dot{w}^{helex_0}_0 - J^helix_1_0_0*\dot{w}^{helex_1}_0 - J^helix_2_0_0*\dot{w}^{helex_2}_0 - J^helix_3_0_0*\dot{w}^{helex_3}_0 + M^airfit_0_0 + M^airfit_1_0 + M^airfit_2_0 + M^airfit_3_0 + M^ext_0)/J^qq_0_0],
[ -((J^helix_0_0_0*w^helix_0_0 + J^helix_1_0_0*w^helix_1_0 + J^helix_2_0_0*w^helix_2_0 + J^helix_3_0_0*w^helix_3_0 + J^qq_0_0*w^qq_0(t))*w^qq_2(t) - (J^helix_0_2_2*w^helix_0_2 + J^helix_1_2_2*w^helix_1_2 + J^helix_2_2_2*w^helix_2_2 + J^helix_3_2_2*w^helix_3_2 + J^qq_2_2*w^qq_2(t))*w^qq_0(t))/J^qq_1_1 + (-J^helix_0_1_1*\dot{w}^{helex_0}_1 - J^helix_1_1_1*\dot{w}^{helex_1}_1 - J^helix_2_1_1*\dot{w}^{helex_2}_1 - J^helix_3_1_1*\dot{w}^{helex_3}_1 + M^airfit_0_

функция Ляпунова для системы

\begin{equation}
 \dot{\vec{X}} = \vec{f(\vec{X})}
\end{equation}

\begin{equation}
 \dot{V} = \nabla V \cdot f(\vec{X})
\end{equation}

In [35]:
"""Символьный ввод функции Ляпунова, через квадратичную форму и энергетический подход"""
Laypunov_func = 1/2 * wqq.T * Jqq * wqq + 1/2 * qq.T * qq + 1/2 * 1/Mqq * dpv.T*dpv

Laypunov_func

Matrix([[0.5*J^qq_0_0*w^qq_0(t)**2 + 0.5*J^qq_1_1*w^qq_1(t)**2 + 0.5*J^qq_2_2*w^qq_2(t)**2 + 0.5*q_0(t)**2 + 0.5*q_1(t)**2 + 0.5*q_2(t)**2 + 0.5*q_3(t)**2 + 0.5*p_0(t)**2/M_qq + 0.5*p_1(t)**2/M_qq + 0.5*p_2(t)**2/M_qq]])

In [36]:
"""переменные по которым берем градиент"""
var = sp.Matrix.vstack(wqq, qq, dpv)
var

Matrix([
[w^qq_0(t)],
[w^qq_1(t)],
[w^qq_2(t)],
[   q_0(t)],
[   q_1(t)],
[   q_2(t)],
[   q_3(t)],
[   p_0(t)],
[   p_1(t)],
[   p_2(t)]])

In [37]:
"""Вычисление градиента функции Ляпунова"""
num = 0
for x in var:
    if num == 0:
        grad = Laypunov_func.diff(x)
        num += 1
    else:
        grad = sp.Matrix.vstack(grad, Laypunov_func.diff(x))
grad

Matrix([
[1.0*J^qq_0_0*w^qq_0(t)],
[1.0*J^qq_1_1*w^qq_1(t)],
[1.0*J^qq_2_2*w^qq_2(t)],
[            1.0*q_0(t)],
[            1.0*q_1(t)],
[            1.0*q_2(t)],
[            1.0*q_3(t)],
[       1.0*p_0(t)/M_qq],
[       1.0*p_1(t)/M_qq],
[       1.0*p_2(t)/M_qq]])

In [38]:
"""правая часть уравнения скалярное произведение функции ляпнова и правых частей системы диффиренциальных уравнений движения"""
lyapR = (grad.dot(right_part_our))
lyapR

1.0*J^qq_0_0*(-(-(J^helix_0_1_1*w^helix_0_1 + J^helix_1_1_1*w^helix_1_1 + J^helix_2_1_1*w^helix_2_1 + J^helix_3_1_1*w^helix_3_1 + J^qq_1_1*w^qq_1(t))*w^qq_2(t) + (J^helix_0_2_2*w^helix_0_2 + J^helix_1_2_2*w^helix_1_2 + J^helix_2_2_2*w^helix_2_2 + J^helix_3_2_2*w^helix_3_2 + J^qq_2_2*w^qq_2(t))*w^qq_1(t))/J^qq_0_0 + (-J^helix_0_0_0*\dot{w}^{helex_0}_0 - J^helix_1_0_0*\dot{w}^{helex_1}_0 - J^helix_2_0_0*\dot{w}^{helex_2}_0 - J^helix_3_0_0*\dot{w}^{helex_3}_0 + M^airfit_0_0 + M^airfit_1_0 + M^airfit_2_0 + M^airfit_3_0 + M^ext_0)/J^qq_0_0)*w^qq_0(t) + 1.0*J^qq_1_1*(-((J^helix_0_0_0*w^helix_0_0 + J^helix_1_0_0*w^helix_1_0 + J^helix_2_0_0*w^helix_2_0 + J^helix_3_0_0*w^helix_3_0 + J^qq_0_0*w^qq_0(t))*w^qq_2(t) - (J^helix_0_2_2*w^helix_0_2 + J^helix_1_2_2*w^helix_1_2 + J^helix_2_2_2*w^helix_2_2 + J^helix_3_2_2*w^helix_3_2 + J^qq_2_2*w^qq_2(t))*w^qq_0(t))/J^qq_1_1 + (-J^helix_0_1_1*\dot{w}^{helex_0}_1 - J^helix_1_1_1*\dot{w}^{helex_1}_1 - J^helix_2_1_1*\dot{w}^{helex_2}_1 - J^helix_3_1_1*\dot{w

In [39]:
"""Производная по времени от функции Ляпунова"""
Laypunov_func.diff(t)

Matrix([[1.0*J^qq_0_0*w^qq_0(t)*Derivative(w^qq_0(t), t) + 1.0*J^qq_1_1*w^qq_1(t)*Derivative(w^qq_1(t), t) + 1.0*J^qq_2_2*w^qq_2(t)*Derivative(w^qq_2(t), t) + 1.0*q_0(t)*Derivative(q_0(t), t) + 1.0*q_1(t)*Derivative(q_1(t), t) + 1.0*q_2(t)*Derivative(q_2(t), t) + 1.0*q_3(t)*Derivative(q_3(t), t) + 1.0*p_0(t)*Derivative(p_0(t), t)/M_qq + 1.0*p_1(t)*Derivative(p_1(t), t)/M_qq + 1.0*p_2(t)*Derivative(p_2(t), t)/M_qq]])

In [40]:
var.diff(t)

Matrix([
[Derivative(w^qq_0(t), t)],
[Derivative(w^qq_1(t), t)],
[Derivative(w^qq_2(t), t)],
[   Derivative(q_0(t), t)],
[   Derivative(q_1(t), t)],
[   Derivative(q_2(t), t)],
[   Derivative(q_3(t), t)],
[   Derivative(p_0(t), t)],
[   Derivative(p_1(t), t)],
[   Derivative(p_2(t), t)]])

In [41]:
laypdt = Laypunov_func.diff(t)
for i in range(10):    
    laypdt = laypdt.subs(var.diff(t)[i], right_part_our[i])
laypdt

Matrix([[1.0*J^qq_0_0*(-(-(J^helix_0_1_1*w^helix_0_1 + J^helix_1_1_1*w^helix_1_1 + J^helix_2_1_1*w^helix_2_1 + J^helix_3_1_1*w^helix_3_1 + J^qq_1_1*w^qq_1(t))*w^qq_2(t) + (J^helix_0_2_2*w^helix_0_2 + J^helix_1_2_2*w^helix_1_2 + J^helix_2_2_2*w^helix_2_2 + J^helix_3_2_2*w^helix_3_2 + J^qq_2_2*w^qq_2(t))*w^qq_1(t))/J^qq_0_0 + (-J^helix_0_0_0*\dot{w}^{helex_0}_0 - J^helix_1_0_0*\dot{w}^{helex_1}_0 - J^helix_2_0_0*\dot{w}^{helex_2}_0 - J^helix_3_0_0*\dot{w}^{helex_3}_0 + M^airfit_0_0 + M^airfit_1_0 + M^airfit_2_0 + M^airfit_3_0 + M^ext_0)/J^qq_0_0)*w^qq_0(t) + 1.0*J^qq_1_1*(-((J^helix_0_0_0*w^helix_0_0 + J^helix_1_0_0*w^helix_1_0 + J^helix_2_0_0*w^helix_2_0 + J^helix_3_0_0*w^helix_3_0 + J^qq_0_0*w^qq_0(t))*w^qq_2(t) - (J^helix_0_2_2*w^helix_0_2 + J^helix_1_2_2*w^helix_1_2 + J^helix_2_2_2*w^helix_2_2 + J^helix_3_2_2*w^helix_3_2 + J^qq_2_2*w^qq_2(t))*w^qq_0(t))/J^qq_1_1 + (-J^helix_0_1_1*\dot{w}^{helex_0}_1 - J^helix_1_1_1*\dot{w}^{helex_1}_1 - J^helix_2_1_1*\dot{w}^{helex_2}_1 - J^helix_3_1

In [42]:
"""Проверка равенства производной от времени и градиента скалярно умноженного на функции дифф уравнения"""
sp.Matrix([lyapR]) - laypdt

Matrix([[0]])

По производной по времени из теоремы Барбашины - Красновского вытекают условия на асимптотическую устойчивость, из которых получим уравнение на управление лопастями.

In [43]:
laypdt[0]

1.0*J^qq_0_0*(-(-(J^helix_0_1_1*w^helix_0_1 + J^helix_1_1_1*w^helix_1_1 + J^helix_2_1_1*w^helix_2_1 + J^helix_3_1_1*w^helix_3_1 + J^qq_1_1*w^qq_1(t))*w^qq_2(t) + (J^helix_0_2_2*w^helix_0_2 + J^helix_1_2_2*w^helix_1_2 + J^helix_2_2_2*w^helix_2_2 + J^helix_3_2_2*w^helix_3_2 + J^qq_2_2*w^qq_2(t))*w^qq_1(t))/J^qq_0_0 + (-J^helix_0_0_0*\dot{w}^{helex_0}_0 - J^helix_1_0_0*\dot{w}^{helex_1}_0 - J^helix_2_0_0*\dot{w}^{helex_2}_0 - J^helix_3_0_0*\dot{w}^{helex_3}_0 + M^airfit_0_0 + M^airfit_1_0 + M^airfit_2_0 + M^airfit_3_0 + M^ext_0)/J^qq_0_0)*w^qq_0(t) + 1.0*J^qq_1_1*(-((J^helix_0_0_0*w^helix_0_0 + J^helix_1_0_0*w^helix_1_0 + J^helix_2_0_0*w^helix_2_0 + J^helix_3_0_0*w^helix_3_0 + J^qq_0_0*w^qq_0(t))*w^qq_2(t) - (J^helix_0_2_2*w^helix_0_2 + J^helix_1_2_2*w^helix_1_2 + J^helix_2_2_2*w^helix_2_2 + J^helix_3_2_2*w^helix_3_2 + J^qq_2_2*w^qq_2(t))*w^qq_0(t))/J^qq_1_1 + (-J^helix_0_1_1*\dot{w}^{helex_0}_1 - J^helix_1_1_1*\dot{w}^{helex_1}_1 - J^helix_2_1_1*\dot{w}^{helex_2}_1 - J^helix_3_1_1*\dot{w

In [44]:
"""разделение выражения на части по слагаемым"""
terms = laypdt[0].as_ordered_terms()

приведение выражения к виду скалярного произведения

In [45]:
"""часть выражения содержащая условия на угловые скорости"""
term_w = sp.Matrix([sp.simplify(terms[0])/wqq[0], sp.simplify(terms[1])/wqq[1], sp.simplify(terms[2])/wqq[2]]) #*wqq скалярно
term_w

Matrix([
[-1.0*J^helix_0_0_0*\dot{w}^{helex_0}_0 - 1.0*J^helix_1_0_0*\dot{w}^{helex_1}_0 - 1.0*J^helix_2_0_0*\dot{w}^{helex_2}_0 - 1.0*J^helix_3_0_0*\dot{w}^{helex_3}_0 + 1.0*M^airfit_0_0 + 1.0*M^airfit_1_0 + 1.0*M^airfit_2_0 + 1.0*M^airfit_3_0 + 1.0*M^ext_0 + 1.0*(J^helix_0_1_1*w^helix_0_1 + J^helix_1_1_1*w^helix_1_1 + J^helix_2_1_1*w^helix_2_1 + J^helix_3_1_1*w^helix_3_1 + J^qq_1_1*w^qq_1(t))*w^qq_2(t) - 1.0*(J^helix_0_2_2*w^helix_0_2 + J^helix_1_2_2*w^helix_1_2 + J^helix_2_2_2*w^helix_2_2 + J^helix_3_2_2*w^helix_3_2 + J^qq_2_2*w^qq_2(t))*w^qq_1(t)],
[-1.0*J^helix_0_1_1*\dot{w}^{helex_0}_1 - 1.0*J^helix_1_1_1*\dot{w}^{helex_1}_1 - 1.0*J^helix_2_1_1*\dot{w}^{helex_2}_1 - 1.0*J^helix_3_1_1*\dot{w}^{helex_3}_1 + 1.0*M^airfit_0_1 + 1.0*M^airfit_1_1 + 1.0*M^airfit_2_1 + 1.0*M^airfit_3_1 + 1.0*M^ext_1 - 1.0*(J^helix_0_0_0*w^helix_0_0 + J^helix_1_0_0*w^helix_1_0 + J^helix_2_0_0*w^helix_2_0 + J^helix_3_0_0*w^helix_3_0 + J^qq_0_0*w^qq_0(t))*w^qq_2(t) + 1.0*(J^helix_0_2_2*w^helix_0_2 + J^helix

In [46]:
terms_ww_0 = (sp.simplify(terms[0])/wqq[0]).as_ordered_terms()
terms_ww_1 = (sp.simplify(terms[1])/wqq[1]).as_ordered_terms()
terms_ww_2 = (sp.simplify(terms[2])/wqq[2]).as_ordered_terms()

In [47]:
term_w_dot_helix_0 = sp.Matrix([terms_ww_0[0], terms_ww_1[0], terms_ww_2[0]]) 
term_w_dot_helix_0

Matrix([
[-1.0*J^helix_0_0_0*\dot{w}^{helex_0}_0],
[-1.0*J^helix_0_1_1*\dot{w}^{helex_0}_1],
[-1.0*J^helix_0_2_2*\dot{w}^{helex_0}_2]])

In [48]:
term_w_dot_helix_1 = sp.Matrix([terms_ww_0[1], terms_ww_1[1], terms_ww_2[1]]) 
term_w_dot_helix_1

Matrix([
[-1.0*J^helix_1_0_0*\dot{w}^{helex_1}_0],
[-1.0*J^helix_1_1_1*\dot{w}^{helex_1}_1],
[-1.0*J^helix_1_2_2*\dot{w}^{helex_1}_2]])

In [49]:
term_w_dot_helix_2 = sp.Matrix([terms_ww_0[2], terms_ww_1[2], terms_ww_2[2]]) 
term_w_dot_helix_2

Matrix([
[-1.0*J^helix_2_0_0*\dot{w}^{helex_2}_0],
[-1.0*J^helix_2_1_1*\dot{w}^{helex_2}_1],
[-1.0*J^helix_2_2_2*\dot{w}^{helex_2}_2]])

In [50]:
term_w_dot_helix_3 = sp.Matrix([terms_ww_0[3], terms_ww_1[3], terms_ww_2[3]]) 
term_w_dot_helix_3

Matrix([
[-1.0*J^helix_3_0_0*\dot{w}^{helex_3}_0],
[-1.0*J^helix_3_1_1*\dot{w}^{helex_3}_1],
[-1.0*J^helix_3_2_2*\dot{w}^{helex_3}_2]])

In [51]:
M_air_0 = sp.Matrix([terms_ww_0[4], terms_ww_1[4], terms_ww_2[4]]) 
M_air_0

Matrix([
[1.0*M^airfit_0_0],
[1.0*M^airfit_0_1],
[1.0*M^airfit_0_2]])

In [52]:
M_air_1 = sp.Matrix([terms_ww_0[5], terms_ww_1[5], terms_ww_2[5]]) 
M_air_1

Matrix([
[1.0*M^airfit_1_0],
[1.0*M^airfit_1_1],
[1.0*M^airfit_1_2]])

In [53]:
M_air_2 = sp.Matrix([terms_ww_0[6], terms_ww_1[6], terms_ww_2[6]]) 
M_air_2

Matrix([
[1.0*M^airfit_2_0],
[1.0*M^airfit_2_1],
[1.0*M^airfit_2_2]])

In [54]:
M_air_3 = sp.Matrix([terms_ww_0[7], terms_ww_1[7], terms_ww_2[7]]) 
M_air_3

Matrix([
[1.0*M^airfit_3_0],
[1.0*M^airfit_3_1],
[1.0*M^airfit_3_2]])

In [55]:
M_ext = sp.Matrix([terms_ww_0[8], terms_ww_1[8], terms_ww_2[8]]) 
M_ext

Matrix([
[1.0*M^ext_0],
[1.0*M^ext_1],
[1.0*M^ext_2]])

In [56]:
term_cross_0 = sp.Matrix([terms_ww_0[9], terms_ww_1[9], terms_ww_2[9]]) 
term_cross_0

Matrix([
[ 1.0*(J^helix_0_1_1*w^helix_0_1 + J^helix_1_1_1*w^helix_1_1 + J^helix_2_1_1*w^helix_2_1 + J^helix_3_1_1*w^helix_3_1 + J^qq_1_1*w^qq_1(t))*w^qq_2(t)],
[-1.0*(J^helix_0_0_0*w^helix_0_0 + J^helix_1_0_0*w^helix_1_0 + J^helix_2_0_0*w^helix_2_0 + J^helix_3_0_0*w^helix_3_0 + J^qq_0_0*w^qq_0(t))*w^qq_2(t)],
[ 1.0*(J^helix_0_0_0*w^helix_0_0 + J^helix_1_0_0*w^helix_1_0 + J^helix_2_0_0*w^helix_2_0 + J^helix_3_0_0*w^helix_3_0 + J^qq_0_0*w^qq_0(t))*w^qq_1(t)]])

In [57]:
term_cross_1 = sp.Matrix([terms_ww_0[10], terms_ww_1[10], terms_ww_2[10]]) 
term_cross_1

Matrix([
[-1.0*(J^helix_0_2_2*w^helix_0_2 + J^helix_1_2_2*w^helix_1_2 + J^helix_2_2_2*w^helix_2_2 + J^helix_3_2_2*w^helix_3_2 + J^qq_2_2*w^qq_2(t))*w^qq_1(t)],
[ 1.0*(J^helix_0_2_2*w^helix_0_2 + J^helix_1_2_2*w^helix_1_2 + J^helix_2_2_2*w^helix_2_2 + J^helix_3_2_2*w^helix_3_2 + J^qq_2_2*w^qq_2(t))*w^qq_0(t)],
[-1.0*(J^helix_0_1_1*w^helix_0_1 + J^helix_1_1_1*w^helix_1_1 + J^helix_2_1_1*w^helix_2_1 + J^helix_3_1_1*w^helix_3_1 + J^qq_1_1*w^qq_1(t))*w^qq_0(t)]])

In [58]:
(term_cross_0+term_cross_1)+wqq.cross(Jqq*wqq + J_helix_0*w_helix_0 + J_helix_1*w_helix_1 + J_helix_2*w_helix_2 + J_helix_3*w_helix_3)

Matrix([
[0],
[0],
[0]])

In [59]:
quat_term = terms[3]+terms[4]+terms[5]+terms[6]
quat_term

1.0*(0.5*q_0(t)*w^qq_0(t) - 0.5*q_2(t)*w^qq_2(t) + 0.5*q_3(t)*w^qq_1(t))*q_1(t) + 1.0*(0.5*q_0(t)*w^qq_1(t) + 0.5*q_1(t)*w^qq_2(t) - 0.5*q_3(t)*w^qq_0(t))*q_2(t) + 1.0*(0.5*q_0(t)*w^qq_2(t) - 0.5*q_1(t)*w^qq_1(t) + 0.5*q_2(t)*w^qq_0(t))*q_3(t) + 1.0*(-0.5*q_1(t)*w^qq_0(t) - 0.5*q_2(t)*w^qq_1(t) - 0.5*q_3(t)*w^qq_2(t))*q_0(t)

In [60]:
quat_term_vec = sp.Matrix([terms[6]/q[0], terms[3]/q[1], terms[4]/q[2], terms[5]/q[3]])
quat_term_vec

Matrix([
[-0.5*q_1(t)*w^qq_0(t) - 0.5*q_2(t)*w^qq_1(t) - 0.5*q_3(t)*w^qq_2(t)],
[ 0.5*q_0(t)*w^qq_0(t) - 0.5*q_2(t)*w^qq_2(t) + 0.5*q_3(t)*w^qq_1(t)],
[ 0.5*q_0(t)*w^qq_1(t) + 0.5*q_1(t)*w^qq_2(t) - 0.5*q_3(t)*w^qq_0(t)],
[ 0.5*q_0(t)*w^qq_2(t) - 0.5*q_1(t)*w^qq_1(t) + 0.5*q_2(t)*w^qq_0(t)]])

In [61]:
right_part_second_equation

Matrix([
[-0.5*q_1(t)*w^qq_0(t) - 0.5*q_2(t)*w^qq_1(t) - 0.5*q_3(t)*w^qq_2(t)],
[ 0.5*q_0(t)*w^qq_0(t) - 0.5*q_2(t)*w^qq_2(t) + 0.5*q_3(t)*w^qq_1(t)],
[ 0.5*q_0(t)*w^qq_1(t) + 0.5*q_1(t)*w^qq_2(t) - 0.5*q_3(t)*w^qq_0(t)],
[ 0.5*q_0(t)*w^qq_2(t) - 0.5*q_1(t)*w^qq_1(t) + 0.5*q_2(t)*w^qq_0(t)]])

In [62]:
right_part_three_equation

Matrix([
[F^grav_0 + F^lift_0*(q_0(t)**2 + q_1(t)**2 - q_2(t)**2 - q_3(t)**2) + F^lift_1*(-2*q_0(t)*q_3(t) + 2*q_1(t)*q_2(t)) + F^lift_2*(2*q_0(t)*q_2(t) + 2*q_1(t)*q_3(t))],
[F^grav_1 + F^lift_0*(2*q_0(t)*q_3(t) + 2*q_1(t)*q_2(t)) + F^lift_1*(q_0(t)**2 - q_1(t)**2 + q_2(t)**2 - q_3(t)**2) + F^lift_2*(-2*q_0(t)*q_1(t) + 2*q_2(t)*q_3(t))],
[F^grav_2 + F^lift_0*(-2*q_0(t)*q_2(t) + 2*q_1(t)*q_3(t)) + F^lift_1*(2*q_0(t)*q_1(t) + 2*q_2(t)*q_3(t)) + F^lift_2*(q_0(t)**2 - q_1(t)**2 - q_2(t)**2 + q_3(t)**2)]])

In [63]:
fors_term_vec = sp.Matrix([terms[7]*Mqq/p[0], terms[8]*Mqq/p[1], terms[9]*Mqq/p[2]])
fors_term_vec

Matrix([
[1.0*F^grav_0 + 1.0*F^lift_0*(q_0(t)**2 + q_1(t)**2 - q_2(t)**2 - q_3(t)**2) + 1.0*F^lift_1*(-2*q_0(t)*q_3(t) + 2*q_1(t)*q_2(t)) + 1.0*F^lift_2*(2*q_0(t)*q_2(t) + 2*q_1(t)*q_3(t))],
[1.0*F^grav_1 + 1.0*F^lift_0*(2*q_0(t)*q_3(t) + 2*q_1(t)*q_2(t)) + 1.0*F^lift_1*(q_0(t)**2 - q_1(t)**2 + q_2(t)**2 - q_3(t)**2) + 1.0*F^lift_2*(-2*q_0(t)*q_1(t) + 2*q_2(t)*q_3(t))],
[1.0*F^grav_2 + 1.0*F^lift_0*(-2*q_0(t)*q_2(t) + 2*q_1(t)*q_3(t)) + 1.0*F^lift_1*(2*q_0(t)*q_1(t) + 2*q_2(t)*q_3(t)) + 1.0*F^lift_2*(q_0(t)**2 - q_1(t)**2 - q_2(t)**2 + q_3(t)**2)]])

In [64]:
fors_term_vec - right_part_three_equation

Matrix([
[0],
[0],
[0]])

In [65]:
k_L = sp.symbols('k_L')
f_lift = k_L * sp.Matrix([[0],[0], [w_helix[2] + w_helix[5] + w_helix[8] + w_helix[11]]])
f_lift

Matrix([
[                                                          0],
[                                                          0],
[k_L*(w^helix_0_2 + w^helix_1_2 + w^helix_2_2 + w^helix_3_2)]])

In [66]:
R_q_f_lift = R_q * f_lift
R_q_f_lift

Matrix([
[            k_L*(2*q_0(t)*q_2(t) + 2*q_1(t)*q_3(t))*(w^helix_0_2 + w^helix_1_2 + w^helix_2_2 + w^helix_3_2)],
[           k_L*(-2*q_0(t)*q_1(t) + 2*q_2(t)*q_3(t))*(w^helix_0_2 + w^helix_1_2 + w^helix_2_2 + w^helix_3_2)],
[k_L*(w^helix_0_2 + w^helix_1_2 + w^helix_2_2 + w^helix_3_2)*(q_0(t)**2 - q_1(t)**2 - q_2(t)**2 + q_3(t)**2)]])

\begin{equation}
V = \frac{1}{2}\vec{\omega}^T \widehat{J}_{qq} \vec{\omega} + \vec{q}^T\vec{q} + \frac{1}{2m_{qq}}\vec{p}^T\vec{p}
\end{equation}

\begin{equation}
  \dot{V} = \left( -\sum_{i} \widehat{J}_{helix,   i} \dot{\vec{\omega}}_{helix, i} + \sum_{j} \vec{M}_{air lift, j} + \vec{M}_{ext} - \vec{\omega}_{qq} \times (\widehat{J}_{qq}\vec{\omega}_{qq} + \sum_{i} \widehat{J}_{helix, i} \vec{\omega}_{helix, i}),  \vec{\omega}_{qq} \right) + \left(\frac{1}{2}\widehat{\Omega} \vec{q}, \vec{q} \right) + \left( \vec{F}_{grav} + \widehat{R}(q(t))\vec{F}_{lift} + \vec{F}_{air}, \frac{\vec{p}}{m_{qq}}\right)\\
\end{equation}

\begin{equation}
V = \frac{1}{2}\vec{\omega}^T \widehat{J}_{qq} \vec{\omega} + k_q(1 - q_{0}) + \frac{1}{2}\vec{p}^T\vec{p}
\end{equation}

\begin{equation}
{\vec{q}}_{vect} = \left( q_1, q_2, q_3 \right)^T
\end{equation}

\begin{equation}
  \dot{V} = \left( -\sum_{i} \widehat{J}_{helix,   i} \dot{\vec{\omega}}_{helix, i} + \sum_{j} \vec{M}_{air lift, j} + \vec{M}_{ext} - \vec{\omega}_{qq} \times (\widehat{J}_{qq}\vec{\omega}_{qq} + \sum_{i} \widehat{J}_{helix, i} \vec{\omega}_{helix, i}),  \vec{\omega}_{qq} \right) + \left(\frac{1}{2} \vec{\omega}_{qq} , k_q{\vec{q}}_{vect} \right) + \left( \vec{F}_{grav} + \widehat{R}(q(t))\vec{F}_{lift} + \vec{F}_{air}, \vec{p}\right)\\
\end{equation}

\begin{equation}
  \dot{V} = \left( -\sum_{i} \widehat{J}_{helix,   i} \dot{\vec{\omega}}_{helix, i} + \sum_{j} \vec{M}_{air lift, j} + \vec{M}_{ext} - \vec{\omega}_{qq} \times (\widehat{J}_{qq}\vec{\omega}_{qq} + \sum_{i} \widehat{J}_{helix, i} \vec{\omega}_{helix, i}) + \frac{1}{2} k_q {\vec{q}}_{vect},  \vec{\omega}_{qq} \right)   + \left( \vec{F}_{grav} + \widehat{R}(q(t))\vec{F}_{lift}, \vec{p}\right)\\
\end{equation}

момент силы создаваемый одним из поднимающих винтов $\vec{M}_{air lift} = \vec{r}\times\ \vec{F}_{lift} $ $ \vec{r}_0 = (\frac{L}{2}, 0, 0) $ $ \vec{r}_1 = (0, \frac{L}{2}, 0) $ $ \vec{r}_2 = (-\frac{L}{2}, 0, 0) $ $ \vec{r}_3 = (0, -\frac{L}{2}, 0) $ а внешнее возмущение, как случайный порыва ветра запишем как  $\vec{M}_{ext}$

подъемная сила винтов для удобства будет выражаться через пропорциональный коэфициент к угловой скорости вращения $\vec{F}_{lift}^i = (0, 0, k_l{\omega_{helix, i}}_z)^T $

\begin{equation}
\vec{M}_{air lift}^0 = 
\begin{vmatrix}
       \vec{i} & \vec{j} & \vec{k} \\
       \frac{L}{2} & 0 & 0 \\
       0           & 0 & k_l{\omega_{helix, 0}}_z
     \end{vmatrix} = 
     \begin{pmatrix}
  0 \\
  -\frac{L}{2} k_l{\omega_{helix, 0}}_z \\
  0
\end{pmatrix}
\end{equation}

\begin{equation}
\vec{M}_{air lift}^1 = 
\begin{vmatrix}
       \vec{i} & \vec{j} & \vec{k} \\
       0 & \frac{L}{2} & 0 \\
       0           & 0 & k_lk_l{\omega_{helix, 1}}_z
     \end{vmatrix} = 
     \begin{pmatrix}
  \frac{L}{2} k_l{\omega_{helix, 1}}_z \\
  0 \\
  0
\end{pmatrix}
\end{equation}

\begin{equation}
\vec{M}_{air lift}^2 = 
     \begin{pmatrix}
  0 \\
  \frac{L}{2} k_l{\omega_{helix, 2}}_z \\
  0
\end{pmatrix}
\end{equation}

\begin{equation}
\vec{M}_{air lift}^3 = 
     \begin{pmatrix}
  -\frac{L}{2} k_l{\omega_{helix, 3}}_z \\
  0 \\
  0
\end{pmatrix}
\end{equation}

управление выведем из теоремы Барбашина - Красовского

\begin{equation}
  \dot{V} \leq 0\\
\end{equation}

\begin{equation}
 -\sum_{i} \widehat{J}_{helix,   i} \dot{\vec{\omega}}_{helix, i} + \sum_{j} \vec{M}_{air lift, j} + \vec{M}_{ext} - \vec{\omega}_{qq} \times (\widehat{J}_{qq}\vec{\omega}_{qq} + \sum_{i} \widehat{J}_{helix, i} \vec{\omega}_{helix, i}) + \frac{1}{2} k_q{\vec{q}}_{vect} = -\gamma \vec{\omega}_{qq}   \\
\end{equation}



\begin{equation}
 \sum_{i} \widehat{J}_{helix,   i} \dot{\vec{\omega}}_{helix, i} = + \sum_{j} \vec{M}_{air lift, j} + \vec{M}_{ext} - \vec{\omega}_{qq} \times (\widehat{J}_{qq}\vec{\omega}_{qq} + \sum_{i} \widehat{J}_{helix, i} \vec{\omega}_{helix, i}) + \frac{1}{2} k_q{\vec{q}}_{vect} + \gamma \vec{\omega}_{qq}   \\
\end{equation}



 что избежать особенносте с поворотами заменим $k_q{\vec{q}}_{vect}$ на $sign(q_0)k_q{\vec{q}}_{vect}$

\begin{equation}
 \sum_{i} \widehat{J}_{helix,   i} \dot{\vec{\omega}}_{helix, i}  = \gamma \vec{\omega}_{qq}  +  \sum_{j} \vec{M}_{air lift, j} + \vec{M}_{ext} - \vec{\omega}_{qq} \times (\widehat{J}_{qq}\vec{\omega}_{qq} + \sum_{i} \widehat{J}_{helix, i} \vec{\omega}_{helix, i}) - \frac{1}{2} sign(q_0)k_q{\vec{q}}_{vect} \\
\end{equation}


\begin{equation}
  \vec{F}_{grav} + \widehat{R}(q(t))\vec{F}_{lift} = -\beta \vec{p}
\end{equation}

разрешим уравнения на управление лопостями

\begin{equation}
 \begin{cases}
  \sum_{j} \vec{M}_{ext, j} + \vec{M}_{air lift}=  \widehat{J}_{qq}\dot{\vec{\omega}}_{qq} + \sum_{i} \widehat{J}_{helix,   i}\dot{\vec{\omega}}_{helix, i} + \vec{\omega}_{qq} \times (\widehat{J}_{qq}\vec{\omega}_{qq} + \sum_{i} \widehat{J}_{helix, i} \vec{\omega}_{helix, i}) \\
   \vec{\dot{q}} = \frac{1}{2}\widehat{\Omega}\vec{q}\\
   \vec{F}_{grav} + \widehat{R}(q(t))\vec{F}_{lift}  = \dot{\vec{p}}\\
 \end{cases}
\end{equation}

\begin{equation}
 \begin{cases}
 \widehat{J}_{qq}\dot{\vec{\omega}}_{qq} = \sum_{j} \vec{M}_{ext, j} + \vec{M}_{air lift} - \sum_{i} \widehat{J}_{helix,   i}\dot{\vec{\omega}}_{helix, i} - \vec{\omega}_{qq} \times (\widehat{J}_{qq}\vec{\omega}_{qq} + \sum_{i} \widehat{J}_{helix, i} \vec{\omega}_{helix, i}) \\
   \vec{\dot{q}} = \frac{1}{2}\widehat{\Omega}\vec{q}\\
   \dot{\vec{p}} = \vec{F}_{grav} + \widehat{R}(q(t))\vec{F}_{lift}\\
 \end{cases}
\end{equation}

\begin{equation}
 \begin{cases}
 \widehat{J}_{qq}\dot{\vec{\omega}}_{qq} = -\gamma \vec{\omega}_{qq} - \frac{1}{2} sign(q_0)k_q{\vec{q}}_{vect}\\
  \sum_{i} \widehat{J}_{helix,   i} \dot{\vec{\omega}}_{helix, i}  = \gamma \vec{\omega}_{qq}  +  \sum_{j} \vec{M}_{air lift, j} + \vec{M}_{ext} - \vec{\omega}_{qq} \times (\widehat{J}_{qq}\vec{\omega}_{qq} + \sum_{i} \widehat{J}_{helix, i} \vec{\omega}_{helix, i}) - \frac{1}{2} sign(q_0)k_q{\vec{q}}_{vect} \\
   \vec{\dot{q}} = \frac{1}{2}\widehat{\Omega}\vec{q}\\
   \dot{\vec{p}} = -\beta \vec{p}\\
   \widehat{R}(q(t))\vec{F}_{lift} = -\beta \vec{p} - \vec{F}_{grav}
 \end{cases}
\end{equation}

или в скалярном виде

\begin{equation}
 \begin{cases}
 \widehat{J}_{qq}\dot{\vec{\omega}}_{qq} = -\gamma \vec{\omega}_{qq} - \frac{1}{2} sign(q_0)k_q{\vec{q}}_{vect}\\
  \sum_{i} \widehat{J}_{helix,   i} \dot{\vec{\omega}}_{helix, i}  = \gamma \vec{\omega}_{qq}  +  \sum_{j} \vec{M}_{air lift, j} + \vec{M}_{ext} - \vec{\omega}_{qq} \times (\widehat{J}_{qq}\vec{\omega}_{qq} + \sum_{i} \widehat{J}_{helix, i} \vec{\omega}_{helix, i}) - \frac{1}{2} sign(q_0)k_q{\vec{q}}_{vect} \\
   \vec{\dot{q}} = \frac{1}{2}\widehat{\Omega}\vec{q}\\
   \dot{\vec{p}} = -\beta \vec{p}\\
   \widehat{R}(q(t))\vec{F}_{lift} = -\beta \vec{p} - \vec{F}_{grav}
 \end{cases}
\end{equation}

\begin{equation}
\sum_{i} \widehat{J}_{helix,   i} \dot{\vec{\omega}}_{helix, i} + \sum_{j} \vec{M}_{air lift, j} + \vec{M}_{ext} - \vec{\omega}_{qq} \times (\widehat{J}_{qq}\vec{\omega}_{qq} + \sum_{i} \widehat{J}_{helix, i} \vec{\omega}_{helix, i}) = -\gamma  \vec{\omega}_{qq} 
\end{equation}

In [369]:
(OmegaTen*qq).dot(qq)

(q_0(t)*w^qq_0(t) - q_2(t)*w^qq_2(t) + q_3(t)*w^qq_1(t))*q_1(t) + (q_0(t)*w^qq_1(t) + q_1(t)*w^qq_2(t) - q_3(t)*w^qq_0(t))*q_2(t) + (q_0(t)*w^qq_2(t) - q_1(t)*w^qq_1(t) + q_2(t)*w^qq_0(t))*q_3(t) + (-q_1(t)*w^qq_0(t) - q_2(t)*w^qq_1(t) - q_3(t)*w^qq_2(t))*q_0(t)

In [361]:
OmegaTen* qq

Matrix([
[-q_1(t)*w^qq_0(t) - q_2(t)*w^qq_1(t) - q_3(t)*w^qq_2(t)],
[ q_0(t)*w^qq_0(t) - q_2(t)*w^qq_2(t) + q_3(t)*w^qq_1(t)],
[ q_0(t)*w^qq_1(t) + q_1(t)*w^qq_2(t) - q_3(t)*w^qq_0(t)],
[ q_0(t)*w^qq_2(t) - q_1(t)*w^qq_1(t) + q_2(t)*w^qq_0(t)]])

In [368]:
QQ = sp.Matrix([[-q[1], -q[2], -q[3]],
                [ q[0],  q[3], -q[2]],
                [-q[3],  q[0],  q[1]],
                [ q[2], -q[1],  q[0]]])
QQ * wqq - OmegaTen* qq

Matrix([
[0],
[0],
[0],
[0]])